In [ ]:
# %matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u
import tqdm 
import matplotlib.animation as anim
from astropy.visualization import astropy_mpl_style, quantity_support
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In this noteboook I keep the old codes that is necesary to rule out for different reasons

In [ ]:
#Playtime data

#Tuple in order of (latitude,longitude)
observer = '4-35-56-N' ,'74-04-51-W'  

#Data Frame with information of the objects, we are working with the three limit cases: Is observable, is not and is near to the horizon 
alert = pd.DataFrame ( {'Name': ['Aldebaran', 'Antares'],
                        'RA': ['04h35m55.64s','16h29m24.17s'],
                        'DEC':['+16d30m27.2s','-26d25m53.56s'] } ) 
 
#Slot of time and date
date_i = '2025-02-17 19:00:00'
date_f = '2025-02-18 07:00:00'

#Time scale of the observation, the minimum is 5 seconds
timescale = ['m', 30]

#Priority items
priority = ['Extraordinary','High','Medium','Low']

#Limit of good observation, usually are 33 degree on altitude parameter
limit = 33

In [ ]:
#Convert the DMS format to degrees
def ConvertLaLo(observer):
    convert = []
    for each in observer:
        ob_ = each.split('-')

        #DMS format
        dms = float(ob_[0]) + (float(ob_[1])/60) + (float(ob_[2])/3600)

        #Change the sign if its on west or sourth
        if ob_[3] == 'W' or ob_[3] == 'S':
            convert.append(dms*(-1))
        else:
            convert.append(dms)

    return convert

alert

#Move the object time in astropy
def MoveTime(time_actual,scale,sc):
    if scale == 's':
        t = time_actual + sc*u.second 
    if scale == 'm':
        t = time_actual + sc*u.minute
    if scale == 'h':
        t = time_actual + sc*u.hour

    return t

In [ ]:
#With time slot and optimized

def Observations(observer, alert, Date_i, Date_f,time_scale):

    #Definition of different variables:
    actual_time = Time(Date_i,format = 'iso', scale='utc')
    final_time = Time(Date_f,format = 'iso', scale='utc')

    lat_conv , lon_conv = ConvertLaLo(observer)
    observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)

    #Lists of data for post-analysis
    Big_Data = []

    #Establish the time slot
    while actual_time <= final_time:

        #For each time slot we need the data of the conditional about observations
        
        #Establish the coordinates with the correction from geocentric calculations of astropy
        celestial_coord = SkyCoord(ra=alert['RA'], dec=alert['DEC']) #Mantain the degrees units

        # Calculate the coordenates AltAz for the time and observer and transform the J2000 coordinate system for altaz
        altaz_coord = celestial_coord.transform_to( AltAz(obstime=actual_time, location=observer) ) 
    
        # Determinate if its observable (altitude > 0 degrees) in the altaz coordinate system, return a boolean.
        state  = altaz_coord.alt > 0*u.deg

        #Put the things together
        alert['Observable'] = state
        alert['Az'] = altaz_coord.az.deg
        alert['Alt'] = altaz_coord.alt.deg
        
        #Put the conditionals of a time slot on a dataframe in the format for better reading
        Data = alert.copy()

        #Eliminate the observations who the state is false
        # Data = Data[Data['Observable'] != False]

        #With the DataFrame we can organice by the higher altitude
        Data = (Data.sort_values(by='Alt', ascending=False, na_position='first')).reset_index(drop=True)

        #Establish the priority by a percent of the list. Because the list is organized by ascending altitude
        p = [priority[0]]*int((98/100)*len(Data['Name'])) + [priority[1]]*int((70/100)*len(Data['Name'])) + [priority[2]]*int((40/100)*len(Data['Name'])) + [priority[3]]*int((20/100)*len(Data['Name']))

        Data['Priority'] = p
        
        #Put the dataframe in a list and the values of time
        Big_Data.append(Data) 

        #Actualize the time depending of the time scale
        actual_time = MoveTime(actual_time,time_scale[0],time_scale[1])

    return Big_Data

Data = Observations(observer, alert, date_i, date_f,timescale)
Data[0]

In [ ]:
#Other functions

#Take the data from the analysis to graphic
def EstablishCoord(Data,i):
    colors=['limegreen','darkorange','midnightblue','hotpink']
    ALT =[]
    AZ = []
    preference = []
    
    for each in Data:
        ALT.append(each['Alt'][i])
        AZ.append(each['Az'][i])

        if each['Priority'][i] == 'Extraordinary':
            preference.append(colors[0])
        
        if each['Priority'][i] == 'High':
            preference.append(colors[1])

        if each['Priority'][i] == 'Medium':
            preference.append(colors[2])

        if each['Priority'][i] == 'Low':
            preference.append(colors[3])

    return ALT,AZ,preference

plt.style.use(astropy_mpl_style)
quantity_support()

for i in range(0,2):
    ALT,AZ,priority = EstablishCoord(Data,i)
    midnight = np.linspace(-2, 20, len(ALT)) 
    plt.scatter(midnight, ALT,color=priority)

plt.show()

In [ ]:
#Take the data from the analysis to graphic
def EstablishCoord(Data):
    ALT = []
    AZ = []
    label = []
    
    for each in Data:
        ALT.append(each['Alt'].iloc[0])
        AZ.append(each['Az'].iloc[0])
        label.append(each['Label'].iloc[0])

    return ALT,AZ,label

ALT,AZ,label = EstablishCoord(Data)

In [ ]:
#Graphic
plt.style.use(astropy_mpl_style)
quantity_support()

plt.scatter(time, ALT,c=AZ,cmap="viridis",lw=1)

#Limits of good observations
plt.plot(time, np.linspace(limit[0],limit[0],len(time)),'--',color='grey')
plt.plot(time, np.linspace(limit[1],limit[1],len(time)),'--',color='grey')

plt.colorbar().set_label("Azimuth [deg]")
plt.xlabel("Hours from Midnight")
plt.ylabel("Altitude [deg]")
plt.title('Starting time {}'.format(date_i))

# Loop for annotation of all points 
for i in range(len(time)): 
    plt.annotate(label[i], (time[i], ALT[i] + 1))

plt.show()

In [ ]:
#Graphic 
plt.style.use(astropy_mpl_style)
quantity_support()

plt.scatter(time, ALT,c=AZ,cmap="viridis",lw=1)

#Limits of good observations
plt.plot(time, np.linspace(limit,limit,len(time)),'--',color='grey')

plt.colorbar().set_label("Azimuth [deg]")
plt.xlabel("Hours around Midnight")
plt.ylabel("Altitude [deg]")
plt.title('Starting time {}'.format(date_i))

plt.scatter([], [], color='limegreen', label='Good')
plt.scatter([], [], color='k', label='Bad')

# Loop for annotation of all points 
for i in range(0,len(order)-1):
    for j in range(0,len(time)):

        if limits[i]<= ALT[j] and limits[i]!=0:
            plt.annotate(label[j], (time[j], ALT[j] + 1),color='limegreen')
        else:
            plt.annotate(label[j], (time[j], ALT[j] + 1),color='k')

plt.legend()
plt.show()


print('Order for observation')
order_alerts

In [ ]:
#Establish the order, priority and limits for good observations
def EstablishLimits(Data,rango,Time):
    graphic = []
    limits = []
    priority = []
    targets = []
    labels = []

    for i in range(1,rango):
        #Organize and add time to the dataframe
        Target = Data.loc[Data['Label'] == i].copy()
        Target['Time'] = Time

        #Add to the list of graphics the dataframe with the values and time
        graph = Target.copy()
        graph = graph[graph['Observable'] != False]
        graphic.append(graph.drop(['Name','RA','DEC'],axis=1))
        
        #Now that we added to the list, do the priority algorimth

        #We want to know the ORDER of the observation, that is determined by the highest altitude
        Target = Target.sort_values(by='Alt', ascending=False, na_position='first')
        targets.append(Target.loc[Target['Label'] == i].head(1))

        #Now we want to caracterize the good observations and give a PRIORITY
        Target['Order max'] = Target['Alt'] > 33.0        
        priority.append( ((Target.head(1)['Alt']).iloc[0],i) )

        #Condition for the ones which dont have an good observation
        condition = (Target[Target['Order max'] != False].tail(1))['Alt']
        if len(condition)==0:
            limits.append(0 )
        else:
            limits.append( condition.iloc[0])

    #Order by the highest altitude
    order = sorted(priority, key=lambda x: x[0], reverse=True)
    order = [each[1] for each in order]

    #Take the index order for organize the dataframe of observations
    indexorder = [int(each-1) for each in order]

    #Order of observations
    order_alerts = pd.concat(targets,axis=0).reindex(indexorder).reset_index(drop=True).drop(['Az','Alt','Time','Observable'],axis=1)

    #Take the values of limits
    limits = [limits[i-1] for i in order]

    #Take the value for labels
    for each in graphic:

        label = each['Label']
        labels.append(label)

    
    return limits, order,order_alerts, graphic, labels